# District Assessment

![school](./school.JPG)

In [75]:
# Packages
import locale as loc
import os
import pandas as pd

In [76]:
# Data File Paths
school_data_file_path = os.path.join(".","schools_complete.csv")
student_data_file_path = os.path.join(".","students_complete.csv")

In [77]:
# Read Data Files
df_school = pd.read_csv(school_data_file_path)
df_student = pd.read_csv(student_data_file_path)

## District Summary

In [78]:
# Count students, students passing math, and students passing reading, for later ratios
student_count = df_school["size"].sum()
pass_math_count = (df_student["math_score"] >= 60).sum()
pass_reading_count = (df_student["reading_score"] >= 60).sum()

# Construct a District Summary DataFrame, by combining scalar summaries from the two raw DataFrames
df_district_summary2 = pd.DataFrame()
df_district_summary2["Total Schools"] = [df_school["School ID"].count()]
df_district_summary2["Total Students"] = student_count
df_district_summary2["Total Budget"] = df_school["budget"].sum()
df_district_summary2["Average Math Score"] = df_student["math_score"].mean()
df_district_summary2["Average Reading Score"] = df_student["reading_score"].mean()
df_district_summary2["Math Pass Rate"] = pass_math_count/student_count
df_district_summary2["Reading Pass Rate"] = pass_reading_count/student_count
df_district_summary2["Overall Pass Rate"] = 0.5 * (pass_math_count + pass_reading_count)/student_count

# Format the DataFrame for Output
df_district_summary2.style.format({
    "Total Students" : "{:,}".format,
    "Total Budget" : "${:,}".format,
    "Average Math Score" : "{:.2f}".format,
    "Average Reading Score" : "{:.2f}".format,
    "Math Pass Rate" : "{:.2%}".format,
    "Reading Pass Rate" : "{:.2%}".format,
    "Overall Pass Rate" : "{:.2%}".format
}).hide_index().set_caption("Table 1:  District Summary")

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
15,"39,170","$24,649,428",78.99,81.88,92.45%,100.00%,96.22%


## School Summary

In [79]:
# Aggregate raw student data by school
df_students_by_school = pd.concat(
    [df_student[["school_name","reading_score","math_score"]],
     (df_student["reading_score"] >= 60).rename("is_reading_passed"),
     (df_student["math_score"] >= 60).rename("is_math_passed")
    ],axis=1, copy=False).groupby("school_name").agg(
        {"reading_score":"mean",
         "math_score":"mean",
         "is_reading_passed":"sum",
         "is_math_passed":"sum"
        }).reset_index(drop=True)

# Select some school data, and sort by school, so this school DataFrame aligns with the student DataFrame
df_school_stats = pd.concat([
    df_school[["school_name","type","size","budget"]].sort_values("school_name",ignore_index=True),
    df_students_by_school], axis=1)

In [91]:
# Blend the simple school statistics with calculated, secondary statistics, for the columns of the school summary
df_school_summary = pd.concat([
    df_school_stats[["school_name","type","size"]], 
    (df_school_stats["budget"]/df_school_stats["size"]).rename("Per Student Budget"),
    df_school_stats[["math_score","reading_score"]],
    (df_school_stats["is_math_passed"]/df_school_stats["size"]).rename("% Passing Math"),
    (df_school_stats["is_reading_passed"]/df_school_stats["size"]).rename("% Passing Reading"),
    (0.5 * (df_school_stats["is_reading_passed"] + df_school_stats["is_math_passed"]) \
     /df_school_stats["size"]).rename("Overall Passing Rate")], axis=1, copy=False)

# Rename columns, for presentation
df_school_summary.rename(columns={"school_name" : "School Name", 
                                  "type" : "School Type",
                                  "size" : "Total Students",
                                  "math_score" : "Average Math Score",
                                  "reading_score" : "Average Reading Score"}, inplace=True)

In [81]:
# Define a reusuable function for styling a school summary table
def style_school_summary(df, caption):
    return df.style.format({
        "Per Student Budget" : "${:,.2f}".format,
        "Average Math Score" : "{:.2f}".format,
        "Average Reading Score" : "{:.2f}".format,
        "% Passing Math" : "{:.2%}".format,
        "% Passing Reading" : "{:.2%}".format,
        "Overall Passing Rate" : "{:.2%}".format
    }).hide_index().set_caption(caption)    

style_school_summary(df_school_summary, "Table 2:  School Summary")

School Name,School Type,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,$628.00,77.05,81.03,89.53%,100.00%,94.76%
Cabrera High School,Charter,1858,$582.00,83.06,83.98,100.00%,100.00%,100.00%
Figueroa High School,District,2949,$639.00,76.71,81.16,88.44%,100.00%,94.22%
Ford High School,District,2739,$644.00,77.10,80.75,89.30%,100.00%,94.65%
Griffin High School,Charter,1468,$625.00,83.35,83.82,100.00%,100.00%,100.00%
Hernandez High School,District,4635,$652.00,77.29,80.93,89.08%,100.00%,94.54%
Holden High School,Charter,427,$581.00,83.80,83.81,100.00%,100.00%,100.00%
Huang High School,District,2917,$655.00,76.63,81.18,88.86%,100.00%,94.43%
Johnson High School,District,4761,$650.00,77.07,80.97,89.18%,100.00%,94.59%
Pena High School,Charter,962,$609.00,83.84,84.04,100.00%,100.00%,100.00%


## Top Performing Schools (By Passing Rate)

In [82]:
# There are more than five schools with overall passing rates of 100%.  
# I chose to use "Average Math Score" as a secondary sort key.
style_school_summary(df_school_summary.sort_values(["Overall Passing Rate","Average Math Score"],ascending=False).head(5),
                    "Table 3:  Top Performing Schools (By Passing Rate)")

School Name,School Type,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Pena High School,Charter,962,$609.00,83.84,84.04,100.00%,100.00%,100.00%
Holden High School,Charter,427,$581.00,83.80,83.81,100.00%,100.00%,100.00%
Wright High School,Charter,1800,$583.00,83.68,83.95,100.00%,100.00%,100.00%
Thomas High School,Charter,1635,$638.00,83.42,83.85,100.00%,100.00%,100.00%
Shelton High School,Charter,1761,$600.00,83.36,83.73,100.00%,100.00%,100.00%


## Bottom Performing Schools (By Passing Rate)

In [83]:
# Use the default ascending sort, to show the bottom five, from worst to 5th worst
style_school_summary(df_school_summary.sort_values("Overall Passing Rate").head(5),
                    "Table 4:  Bottom Performing Schools (By Passing Rate)")

School Name,School Type,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Figueroa High School,District,2949,$639.00,76.71,81.16,88.44%,100.00%,94.22%
Rodriguez High School,District,3999,$637.00,76.84,80.74,88.55%,100.00%,94.27%
Huang High School,District,2917,$655.00,76.63,81.18,88.86%,100.00%,94.43%
Hernandez High School,District,4635,$652.00,77.29,80.93,89.08%,100.00%,94.54%
Johnson High School,District,4761,$650.00,77.07,80.97,89.18%,100.00%,94.59%


## Math Scores by Grade

In [84]:
# Collect pertinent student data from the raw DataFrame.
# Replace "9th" grade with "09th" grade, in a new series named "grade_sort" (for correct alphanumerical sorting)
df_scores_school_grade = pd.concat([df_student[["school_name","math_score","reading_score"]], 
                                    df_student["grade"].str.replace("9","09").rename("grade_sort")],axis=1)

# Pivot the scores, by school (index) grade (columns), for both math and reading
df_pivoted_scores = pd.pivot_table(
    df_scores_school_grade, 
    index="school_name", 
    columns="grade_sort", 
    values=["math_score","reading_score"],
    aggfunc="mean").reset_index()

# Replace "09th" grade with "9th" grade, for aesthetics
df_pivoted_scores.rename(columns={"09th" : "9th"})

# Present the math scores
df_pivoted_scores[["school_name","math_score"]].style.format("{:.2f}", subset=pd.IndexSlice[:,'math_score']) \
    .hide_index().set_caption("Table 5:  Math Scores, by School and Grade")

## Reading Scores by Grade

In [85]:
# Present the reading scores
df_pivoted_scores[["school_name","reading_score"]].style.format("{:.2f}", subset=pd.IndexSlice[:,'reading_score']) \
    .hide_index().set_caption("Table 6:  Reading Scores, by School and Grade")

## Scores by Per Student Spending

In [86]:
# The final three "binned" reports are similar.  Build a single sequence of DataFrames with the bins for each report

# Determine the reasonble bins for school spending by student
df_school_summary["Per Student Budget"].sort_values()

# Visually, I see an range near $100.  I will select [bottom,top) bins, to best distribute the schools into bins
#[575,600),[600,625),[625,650),[650,675)
spending_bins = pd.cut(df_school_summary["Per Student Budget"],range(575,700, 25),right=False).rename("Budget Range")

# Determine the reasonable bins for school size
#[0,1750,3500,5250]
size_bins = pd.cut(df_school_summary["Total Students"],
                   range(0, 7000, 1750),
                   right=True,
                   labels=["Small","Medium","Large"]).rename("Size Range")

# Determine the bins for school type
# Not necessary; the df_school_summary DataFrame already has the School Type column

# Here are the school and their bins (this intermediate data is shown only for clarity)
pd.concat([df_school_summary["School Name"], spending_bins, size_bins, df_school_summary["School Type"]], axis = 1)

,School Name,Budget Range,Size Range,School Type
0,Bailey High School,"[625, 650)",Large,District
1,Cabrera High School,"[575, 600)",Medium,Charter
2,Figueroa High School,"[625, 650)",Medium,District
3,Ford High School,"[625, 650)",Medium,District
4,Griffin High School,"[625, 650)",Small,Charter
5,Hernandez High School,"[650, 675)",Large,District
6,Holden High School,"[575, 600)",Small,Charter
7,Huang High School,"[650, 675)",Medium,District
8,Johnson High School,"[650, 675)",Large,District
9,Pena High School,"[600, 625)",Small,Charter


In [93]:
# The average scores and average passing rates should be student-weighted across the bins
#     (Equation 1)      weighted_avg_per_bin = sigma(students_per_school * scores)/students_per_bin
# This DataFrame represents the "(students_per_school * scores)" terms of the numerator:
df_with_bins_weighted = pd.concat(
    [spending_bins,
     size_bins,
     df_school_summary[["School Type", "Total Students"]],
     df_school_summary[["Average Math Score", 
                        "Average Reading Score",
                        "% Passing Math",
                        "% Passing Reading",
                        "Overall Passing Rate"]].multiply(df_school_summary["Total Students"],axis="index")],
    axis=1, copy=False)

# Create a additional column, that will count the schools per bin.  I think this is an important metric to add to the report.
df_with_bins_weighted["School Count"] = 1

# From this binned, weighted Data Frame, we can group, aggregate, and scale the metrics, for the three reports.
# Define a common function to generate each report
def report_scores_by_bin(df_weighted, bin_name, caption):
    # Group and sum (aggregate) the weighted metrics for the report.
    # This DataFrame represents the numerator of Equation 1.
    df_grouped_by_bins = df_weighted.groupby(bin_name).agg("sum").reset_index()

    # Finish Equation 1
    df_bin_report = pd.concat([
        df_grouped_by_bins[[bin_name, "School Count"]],
        df_grouped_by_bins[["Average Math Score", 
                            "Average Reading Score",
                            "% Passing Math",
                            "% Passing Reading",
                            "Overall Passing Rate"]].divide(df_grouped_by_bins["Total Students"],axis="index")],
        axis=1, copy=False)
    
    # Present the styled report
    return df_bin_report.style.format({
        "Average Math Score" : "{:.2f}".format,
        "Average Reading Score" : "{:.2f}".format,
        "% Passing Math" : "{:.2%}".format,
        "% Passing Reading" : "{:.2%}".format,
        "Overall Passing Rate" : "{:.2%}".format
    }).hide_index().set_caption(caption)

In [88]:
report_scores_by_bin(df_with_bins_weighted, "Budget Range", "Table 7:  Scores by Spending Per Student")

Budget Range,School Count,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
"[575, 600)",4,83.36,83.96,100.00%,100.00%,100.00%
"[600, 625)",2,83.53,83.84,100.00%,100.00%,100.00%
"[625, 650)",6,78.06,81.43,90.92%,100.00%,95.46%
"[650, 675)",3,77.05,81.01,89.07%,100.00%,94.53%


## Scores by School Size

In [89]:
report_scores_by_bin(df_with_bins_weighted, "Size Range", "Table 8:  Scores by School Size")

Size Range,School Count,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Small,4,83.52,83.88,100.00%,100.00%,100.00%
Medium,7,79.89,82.40,94.12%,100.00%,97.06%
Large,4,77.07,80.93,89.11%,100.00%,94.56%


## Scores by School Type

In [90]:
report_scores_by_bin(df_with_bins_weighted, "School Type", "Table 9:  Scores by School Type")

School Type,School Count,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Charter,8,83.41,83.90,100.00%,100.00%,100.00%
District,7,76.99,80.96,89.03%,100.00%,94.52%


## Observable Trends
* Reading scores generally exceed math scores
* Per student spending and school size are both negatively correlated with student performance
* Charter schools recorded higher scores than district schools